<a href="https://colab.research.google.com/github/MatteoAldovardi92/Taxi_Project/blob/main/TaxiDriverClustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!pip install -q kaggle

!kaggle datasets download -d mnavas/taxi-routes-for-mexico-city-and-quito

!unzip taxi-routes-for-mexico-city-and-quito.zip



ValueError: mount failed